# Recoding Voice

In [17]:
# # 필요한 패키지
# ! pip install --quiet git+https://github.com/openai/whisper.git
# ! pip install --quiet sounddevice
# ! pip install --quiet ipywebrtc
# ! pip install --quiet gradio
# ! pip install --quiet ipywidgets
# ! pip install --quiet jupyterlab-widgets
# ! jupyter nbextension enable --py widgetsnbextension
# ! apt install --quiet ffmpeg

In [61]:
# 음성 녹음 기능을 사용하기 위해서는 
# ffmpeg 설정을 해야 사용 가능합니다.

In [6]:
from ipywebrtc import CameraStream, AudioRecorder
import openai
import os

In [7]:
with open('../sample_data/token.txt', 'r') as file:
    os.environ["OPENAI_API_KEY"] = file.read().strip()

In [8]:
openai.api_key = os.getenv("OPENAI_API_KEY")

In [21]:
camera =  CameraStream(constraints={'audio': True,'video':False})
recorder = AudioRecorder(stream=camera)

In [22]:
recorder

AudioRecorder(audio=Audio(value=b'', format='webm'), stream=CameraStream(constraints={'audio': True, 'video': …

In [24]:
# 녹음된 음성을 파일로 저장합니다
with open('recording.webm', 'wb') as f:
    f.write(recorder.audio.value)

In [25]:
import ffmpeg

In [34]:
input_file = 'recording.webm'
output_file = 'customer_recording.wav'
ffmpeg_bin = r'ffmpeg.exe'

In [33]:
(
    ffmpeg
    .input(input_file)
    .output(output_file, ac=1, format='wav')
    .run(cmd=ffmpeg_bin, overwrite_output=True, quiet=True)
)

(b'',
 b"ffmpeg version 2024-08-07-git-94165d1b79-full_build-www.gyan.dev Copyright (c) 2000-2024 the FFmpeg developers\r\n  built with gcc 13.2.0 (Rev5, Built by MSYS2 project)\r\n  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-libxml2 --enable-gmp --enable-bzlib --enable-lzma --enable-libsnappy --enable-zlib --enable-librist --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-libbluray --enable-libcaca --enable-sdl2 --enable-libaribb24 --enable-libaribcaption --enable-libdav1d --enable-libdavs2 --enable-libopenjpeg --enable-libquirc --enable-libuavs3d --enable-libxevd --enable-libzvbi --enable-libqrencode --enable-librav1e --enable-libsvtav1 --enable-libvvenc --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxavs2 --enable-libxeve --enable-libxvid --enable-libaom --enable-libjxl --enable-libvpx --enable-mediafoundation --enable-liba

# Converting STT (speech to text)

In [35]:
client = openai.OpenAI()

In [36]:
audio_file= open("customer_recording.wav", "rb")
transcription = client.audio.transcriptions.create(
  model="whisper-1",
  file=audio_file
)

In [37]:
print(transcription.text)

사는게 참 힘든데 내가 참 고등학교를 돌아간다면은 공부를 좀 더 열심히 할 것 같아


# Moderating

In [38]:
response = client.moderations.create(input=transcription.text)
output = response.results[0]
output.to_dict()

{'categories': {'harassment': False,
  'harassment/threatening': False,
  'hate': False,
  'hate/threatening': False,
  'self-harm': False,
  'self-harm/instructions': False,
  'self-harm/intent': False,
  'sexual': False,
  'sexual/minors': False,
  'violence': False,
  'violence/graphic': False},
 'category_scores': {'harassment': 0.00019226067524868995,
  'harassment/threatening': 2.1050480427220464e-05,
  'hate': 1.7699398085824214e-05,
  'hate/threatening': 1.2322320230850892e-07,
  'self-harm': 8.603269634477329e-06,
  'self-harm/instructions': 4.030248419439886e-06,
  'self-harm/intent': 4.462227298063226e-05,
  'sexual': 0.00021022817236371338,
  'sexual/minors': 9.107738151215017e-05,
  'violence': 0.0002738391049206257,
  'violence/graphic': 1.2220264579809736e-05},
 'flagged': False}

# Generating Answer

In [9]:
# @title
from openai import OpenAI
import json
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [10]:
client = OpenAI()

In [11]:
# text-embedding-ada 불러서 임베딩 가져오는 펑션
def get_embeddings(text):
  return client.embeddings.create(
    model="text-embedding-ada-002",
    input=text,
    encoding_format="float"
  ).data[0].embedding

In [12]:
# 임베딩을 만들어봅시다!
no_mon_embed      = get_embeddings("돈이 없다")
timetravel_embed  = get_embeddings("회귀")
bodysnat_embed    = get_embeddings("빙의")
reincarn_embed    = get_embeddings("환생")

In [13]:
# 미니 VectorDB 를 만들어봅시다.. --> 미리 카테고리화
solutions ={
    "돈이 없다": {
        "embeddings": no_mon_embed,
        "solutions": ["억만장자가 된다!"]
    },
    "회귀": {
        "embeddings": timetravel_embed,
        "solutions": ["시간을 거슬러 되돌아 감!"]
    },
    "빙의": {
        "embeddings": bodysnat_embed,
        "solutions": ["소설속 캐릭터로 빙의하자!"]
    },
    "환생": {
        "embeddings": reincarn_embed,
        "solutions": ["돈 많고 시간이 많은 캐릭터로 다시 태어나자!"]
    }
}

In [52]:
def get_highest_match(problem_embed):
    # 사용자의 문제를 받고, 우리의 미니 DB 에서 제일 가까운 솔루션을 찾아낼 것입니다!
    results = []
    for key, value in solutions.items():
        sim = cosine_similarity([problem_embed], [value['embeddings']])[0][0]
        results.append((key, sim, value['solutions']))

    # 유사도로 정리
    results.sort(key=lambda x: x[1], reverse=True)

    # 제일 유사한 하나만 가져오기
    return results[0]

In [53]:
def get_solution(problem) -> str:
    # 사용자의 문제를 임베딩으로 바꿈
    problem_embed = get_embeddings(problem)

    # 제일 가까운 카테고리의 솔루션을 가지고 옴
    solution_found = get_highest_match(problem_embed)[2][0]
    return solution_found

In [54]:
def get_problem(user_query):
  # 사용자의 질문을 듣고 카테고리를 선별함
  return client.chat.completions.create(
        model="gpt-4",
        temperature=0,
        messages=[
            {"role": "system", "content": """사용자의 질문을 듣고 그 문제를 해결할 수 있는 방법을 다음 다섯가지 중에서 선택해줘:
            '돈이 없다', '회귀', '빙의', '환생'
            이 중에 선택할 옵션이 없으면 그냥 아무 대답도 하지 마.
            """
            },
            {"role": "user", "content": user_query},
        ]
    ).choices[0].message.content

In [56]:
def get_detailed_solution(user_query):
  # 모든 자료가 모였으니 이제 마지막 답변을 생성함
  solution = get_solution(get_problem(user_query))
  return client.chat.completions.create(
        model="gpt-4",
        temperature=0,
        messages=[
            {"role": "system", "content": """너는 이세계 스토리 작가 AI이다. 너의 역할은 주변에서 흔히 일어날 수 있는 문제를 가진 사람의 불평을 기본으로,
            이세계로 이동시켜서 불평을 잠재우는 설정을 만들어내는 것이다. 이 스토리 안에서 너는 현대인이 길에서 발견하는 작은 디바이스이며, 
            그 사람이 무언가에 대해 불평하면 그 이세계로 이동시켜서 불평을 잠재우는 설정을 제안하면 된다.

            예:
            사람: "아, 난 이번 생은 망한 것 같아."
            AI: "그렇다면 이세계로 환생하는 것은 어떨까요? 부유한 백작가의 막내 아들로 말이죠!

            """
            },
            {"role": "user", "content": user_query},
            {"role": "system", "content": f"""
            이번 사용자의 불평은 {user_query} 이며, 이것에 대한 해결법은 {solution}이다.
            이전의 예시를 참고하여 사용자에게 그럴듯한 이세계로 이동시켜서 불평을 잠재우는 설정 제안해라.
            이세계의 설정에는 가상의 나라, 그리고 황족, 왕족, 귀족 등의 신분, 기사나 사제, 상단주 혹은 마법사 같은 직업이 필요하다.
            머리색깔과 눈색깔도 여러가지로 설정할 수 있다. 예: 빨강, 검정, 금색, 은색, 파랑, 녹색, 보라색, 자주색, 갈색.
            주인공이 사는 가상의 나라와, 가상의 신분과, 가상의 직업과, 머리색깔과 눈색깔을 정하고, 신분과 직업에 기반한 스토리 라인을 정한다.
            기본적으로 시작할 때에는 불행하나 훌륭한 능력과 운으로 크게 성공한다는 줄거리로 200자 이상의 내용을 만들어 보낸다.

            이세계로 이동시켜서 불평을 잠재우는 설정:

            """
            },
        ]
    ).choices[0].message.content

In [57]:
answer = get_detailed_solution(transcription.text)

In [58]:
print(answer)

"그럼 어떠한가요? 고대의 지식을 가진 마법사로 이세계로 이동하는 것은. 그곳은 '아르카디아'라는 이름의 평화로운 나라입니다. 당신은 그곳에서 '아이리스'라는 이름의 귀족으로 태어나게 됩니다. 당신의 머리색깔은 은색이며, 눈은 묘한 푸른색입니다. 

당신은 아이리스 귀족의 막내 아들로 태어나, 고등학교 시절의 지식을 가지고 있어서, 그 나라에서는 누구보다도 뛰어난 학식을 자랑합니다. 당신은 그 지식을 이용해 마법사로서의 길을 걷게 되며, 그 능력은 누구도 따라올 수 없는 수준이 됩니다. 

당신의 능력은 왕국을 구하는데 큰 역할을 하게 되고, 당신은 그 공로로 왕국의 최고 마법사로 승격되게 됩니다. 그리고 당신은 그 지위를 이용해 왕국의 교육 체제를 개혁하게 되며, 그 결과로 모든 사람들이 공부에 열중하게 됩니다. 

당신의 능력과 노력으로 인해, 아르카디아는 지식과 학문의 나라로 발전하게 되고, 당신은 그 나라에서 가장 존경받는 인물이 됩니다. 이렇게 당신은 고등학교 시절의 후회를 바탕으로, 새로운 세상에서 큰 성공을 거두게 됩니다."


# TTS (Converting)

In [59]:
speech_file_path = "speech_output.mp3"
response = client.audio.speech.create(
  model="tts-1",
  voice="nova",
  input=answer
)
response.stream_to_file(speech_file_path)

C:\Users\User\AppData\Local\Temp\ipykernel_32452\706516023.py:7: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  response.stream_to_file(speech_file_path)


In [60]:
from IPython.display import Audio, display
Audio("speech_output.mp3")  # From URL